In [1]:
from RFSM_python.RFSM_funtions import *

# INPUT

In [2]:
path_project = '/Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/MURCIA/RFSM/'
#DTM=path_project+'ascii/dtm.asc'
#COTA_ESTUDIO = 10
path_output = path_project+'/ascii/'
#CoastLine=path_project+'shp/CoastPolygon.shp'
DTM_CLIP_LC = '/Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/MURCIA/RFSM/DATA_IN/MDT/mdt.tif'
path_install_RFSM = '/Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/IHRFSM_v1.0/'

In [3]:
#path_inputs_dinamicas='E:/RFSM/CASO_ASTURIAS/DINAMICAS_ASTURIAS/MARINE_DYNAMICS/'
n_periods = 13

In [4]:
TestID = 1
TestDesc = 'Murcia_Malla_1'
BCSetID = 1
StartTime = '0#h'
EndTime = str(n_periods)+'#h'
TimeStep = 1
SaveTimeStep = '20#m'
MaxTimeStep = 20
MinTimeStep = 0.0001
AlphaParameter = 0.7
ManningGlobalValue = 0.035
Results = '1'
make_dirs(path_install_RFSM,path_project,TestDesc,Results)

# PROCESO

## IMPACT ZONES

In [5]:
Table_impact_zone, izcoast = impact_zones_process(path_project,DTM_CLIP_LC,cellsize=5,new_coast_Line = True)

Unable to open /Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/MURCIA/RFSM/ascii/check/izid1.asc


ERROR 4: /Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/MURCIA/RFSM/ascii/check/izid1.asc: No such file or directory
ERROR 4: /Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/MURCIA/RFSM/ascii/check/izid2.asc: No such file or directory


Unable to open /Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/MURCIA/RFSM/ascii/check/izid2.asc


DriverError: /Users/salvadornavasfernandez/Documents/GitHub/RFSM/tests/MURCIA/RFSM/shp/DTM_OC.shp: No such file or directory

In [127]:
izcoast_modify(path_project+'shp/izcoast.shp',Table_impact_zone, path_project+'ascii/izcoast.csv')

## Puntos de entrada

In [128]:
Table_impact_zone=pd.read_csv(path_project+'ascii/tblImpactZone.csv',index_col=0)
izcoast=pd.read_csv(path_project+'ascii/izcoast.csv',index_col=0)

###  LEVEL IN A SERIES OF POINTS ALONG THE COAST

In [129]:
# Input 2: temporal
CF_FV = 1
xD = 569590.015
yD = 3110298.838
CM = 2.8
CI_pleamar = [1.6, 2, 2.4, 2.8, 2.4, 2, 1.6]
#CI_pleamar = [1.6, 2, 3,4, 3, 2, 1.6]

Tp = 22;
Hs = 8;
Dur = 10;

# Input 1: pleamar   


In [130]:
Table_impact_zone['IZRunOffCoef'] = 0.6
Table_impact_zone['RainfallZoneID'] = 1

BCTypeID_COAST = 2

In [131]:
tusrBCFlowLevel (path_project,TestDesc,Table_impact_zone,izcoast,BCSetID,BCTypeID_COAST,
                    CF_FV,xD,yD,CM,CI_pleamar, n_periods,input_=2)

100%|███████████████████████████████████████████████████████████████████████████████| 590/590 [00:03<00:00, 161.73it/s]


## Manning

In [132]:
create_file_manning(path_project,'F:/PIMA_ADAPTA_MURCIA/GIS/rugosidad',TestDesc,Table_impact_zone)

100%|███████████████████████████████████████████████████████████████████████████| 23983/23983 [03:25<00:00, 116.69it/s]


## Create XML

In [133]:
create_xml(TestID,TestDesc,BCSetID,StartTime,EndTime,TimeStep,SaveTimeStep,MaxTimeStep,MinTimeStep,AlphaParameter,ManningGlobalValue,path_project,Results)

## Ejecución 

In [134]:
execute_RFSM(path_project,TestDesc,TestID)

Assuming windows!


## Leer y exportar resultados

In [135]:
export_result_RFSM_2(path_project,TestDesc,Results)

100%|██████████████████████████████████████████████████████████████████████████| 23983/23983 [00:19<00:00, 1228.62it/s]
